In [1]:
%load_ext nb_black
import pathlib
import sys

import numpy as np
from itertools import permutations
import pandas as pd

sys.path.append("/Users/darryl/proj/carbonplan/retro/")

from retrospective.load.project_db import (
    load_project_db,
)  # this could be better organized.
from retrospective.load.issuance import load_issuance_table
from retrospective.analysis import allocation

<IPython.core.display.Javascript object>

This notebook is a first attempt at building the retro-db, containing all values extracted from Offset Project Data Reports (OPDRs) and marrying that data with the September 9, 2020 ARB issuance table. `retro-db` feeds into numerous downsteram analyses, including our initial efforts to lightly audit ARBOC issuance to individual projects.


Means we end up with three "allocation" values:
- Issuance: the official, issued allocation of ARBOCs as recorded by ARB.
- OPDR-Reported: the OPO/APD reported ARBC
- OPDR-Calculated: issuance derived from IFM-1, IFM-3, IFM-7, IFM-8, and secondary effects (SE). 

In a separate notebook, we explore cases where these values diverge.


# Load retro-db and issuance table

In [2]:
project_db = load_project_db("Forest-Offset-Projects-v0.3", use_cache=True)

loading load Forest-Offset-Projects-v0.3 from /Users/darryl/proj/carbonplan/retro/data


<IPython.core.display.Javascript object>

There is this strange sub-class of projects within the compliance market which we refer to as "graduated" projects -- these are projects that started out in the "Early Action" period and later "graduated" into the full-fledged compliance program. 
Unfortunately, these projects tend to be materially deficient and tough to work with -- there are all sorts of issues with getting their numbers correct. 


In [3]:
EXCLUDE_GRADUATED_PROJECTS = True

if EXCLUDE_GRADUATED_PROJECTS:
    project_db = project_db[
        ~project_db["project"]["early_action"].str.startswith("CAR")
    ]
    project_db = project_db[
        project_db["baseline"]["initial_carbon_stock"]
        > project_db["baseline"]["common_practice"]
    ]

<IPython.core.display.Javascript object>

## Get Issuance

In [4]:
# TODO: store this issuance file somewhere else!
issuance_table = load_issuance_table(
    "/Users/darryl/forest-retro/documents-of-interest/arb/issuance/arboc_issuance_2020-09-09.xlsx"
)

# One project has multiple issuance events in its first reporting period, aggregate them
agg_by_rp = issuance_table.groupby(["opr_id", "arb_rp_id"])[
    ["allocation", "buffer_pool"]
].sum()
issuance_first_rp = agg_by_rp.xs("A", level=1)

<IPython.core.display.Javascript object>

## OPDRcalculated
OPDRs report five individual components that we use to recalculate ARBOC issuance:

- IFM-1: standing live
- IFM-3: standing dead
- IFM-7: in-use wood products
- IFM-8: landfilled wood products
- Secondary Effects: market leakage &etc.

We use these five "components", as reported in the OPDR for both the Baseline scenario (imaginary/counterfactual) and the Project scenario (what actually happened), to re-derive the ARBOC allocation. 
This step (i) gives us confidence in the integrity of our data entry and (ii) lays the foundation for *re-calculating* ARBOCs under different common practice scenarios (see Notebook TK)

And it turns out that joining to a multi-index is impossible....

I have now tried: 
- .loc assigning doesnt work; nor does just naive df['foo']['bar'] = 'baz'
- then i tried joins project_db['rp_1'].join(opdr_calculated). The join would work but I wasn't able to get the result to 'stick'. 
- then i tried adding a multicolumn index to opdr_calculated and doing the join. Here, again, the join is fine (no warning) but I cannot assign the result to the project_db object. There is some black magic here I don't understand. Starting to wonder if this nested-ness is useful or just getting in the way. 


In [5]:
opdr_calculated = allocation.calculate_allocation(project_db, round_intermediates=False)


opdr_calculated = opdr_calculated.to_frame()

opdr_calculated.columns = pd.MultiIndex.from_product(
    [["allocation"], opdr_calculated.columns]
)

<IPython.core.display.Javascript object>

In [6]:
project_db["rp_1"] = project_db["rp_1"].join(opdr_calculated)
project_db["rp_1"] = project_db["rp_1"].join(
    issuance_first_rp["allocation"].rename(
        "arb_issuance"
    )  # TODO: deal with buffer pool stuff later
)

/Users/darryl/py37/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


<IPython.core.display.Javascript object>

In [11]:
opr_id = "ACR276"
opdr_calculated.loc[opr_id] - project_db["rp_1"]["allocation"].loc[opr_id]

allocation  opdr_calculated    3298.4
Name: ACR276, dtype: float64

<IPython.core.display.Javascript object>

In [45]:
project_db["rp_1"] = project_db["rp_1"].join(opdr_calculated,)

<IPython.core.display.Javascript object>

In [46]:
project_db["rp_1"]  # doesnt work either...[('allocation', 'calculated')]

date_submitted                 start                   end  \
                                                                             
CAR1205   2020-04-30T00:00:00Z  2017-12-19T00:00:00Z  2018-08-18T00:00:00Z   
VCSOPR10  2018-05-21T00:00:00Z  2015-10-30T00:00:00Z  2017-10-29T00:00:00Z   
ACR192    2014-11-20T00:00:00Z  2013-05-13T00:00:00Z  2013-12-06T00:00:00Z   
ACR248    2016-08-31T00:00:00Z  2015-06-24T00:00:00Z  2016-03-23T00:00:00Z   
ACR247    2019-08-28T00:00:00Z  2014-05-16T00:00:00Z  2016-05-15T00:00:00Z   
...                        ...                   ...                   ...   
CAR1103                   None  2014-06-21T00:00:00Z  2015-06-21T00:00:00Z   
ACR292    2017-05-02T00:00:00Z  2015-10-21T00:00:00Z  2016-06-30T00:00:00Z   
ACR211                    None  2012-09-19T00:00:00Z  2014-07-31T00:00:00Z   
CAR1032   2015-01-22T00:00:00Z  2013-04-25T00:00:00Z  2013-12-31T00:00:00Z   
ACR324    2018-03-06T00:00:00Z  2016-06-10T00:00:00Z  2016-12-09T00:00:00Z   

         reversal   allocation buffer_contribution confidence_deduction  \
                                                                          
CAR1205            2.95903e+06              569318                    0   
VCSOPR10                 99139               18489                    0   
ACR192                  201277               38726                0.041   
ACR248             2.95262e+06              568084                    0   
ACR247                  780167              150105                    0   
...           ...          ...                 ...                  ...   
CAR1103                 274079               52623                 0.02   
ACR292                  117216               22505                    0   
ACR211             4.45164e+06              626973                    0   
CAR1032                 193277               37110                    0   
ACR324             1.10623e+07             1755591               0.0397   

           components                                                 \
                ifm_1        ifm_3    ifm_7    ifm_8   ifm_14 ifm_17   
CAR1205   7.81476e+06       129401        0        0   -27304      0   
VCSOPR10       322193         5522        0        0      NaN    NaN   
ACR192         492518      5956.63        0        0      NaN    NaN   
ACR248    1.28497e+07       104096  3309.75  4241.17 -31140.4    NaN   
ACR247    7.01274e+06       195576  26874.1  34328.9        0    NaN   
...               ...          ...      ...      ...      ...    ...   
CAR1103        542167         7352        0        0      NaN    NaN   
ACR292    1.28744e+06        17380        0        0    -2126    NaN   
ACR211    1.09752e+07       988165        0        0        0      0   
CAR1032   1.29158e+06        27504      351      621      NaN    NaN   
ACR324    3.36953e+07  5.84137e+06        0        0 -43453.8    NaN   

         secondary_effects            attestation         \
                                             name is_opo   
CAR1205             -27304           David Hoffer   True   
VCSOPR10              -900         Roger Williams  False   
ACR192            -1594.77           Hunter Parks   True   
ACR248            -31140.4  Kaarsten Turner Dalby   True   
ACR247                   0        Kaarsten Turner  False   
...                    ...                    ...    ...   
CAR1103              -2416      Brian Shillinglaw  False   
ACR292               -2126         James D. Clark  False   
ACR211              -49614        Jonathan Brooks   True   
CAR1032              -3381  Kaarsten Turner Dalby   True   
ACR324            -43453.8      Brian L Kleinhenz   True   

                                                      notes  
                                                             
CAR1205                                                      
VCSOPR10  Alllocation differs from issuance table; no ex...  
ACR192                                                     

<IPython.core.display.Javascript object>